In [1]:
!pip install keras

    100% |################################| 307kB 3.0MB/s ta 0:00:01
    100% |################################| 276kB 3.2MB/s ta 0:00:01
    100% |################################| 51kB 8.3MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
Successfully built pyyaml
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [60]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [5]:
df = pd.read_csv('storage/haikus.csv', encoding='latin1')
df = df[df['lang'] == 'en'].copy()
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,haiku,lang
3,3,Memorial Day --,a shadow for each,white cross,NaN,NaN,NaN,NaN,NaN,NaN,Memorial Day --\r\na shadow for each\r\nwhite ...,en
4,4,spring rain -,as the doctor speaks,i think of lilacs,NaN,NaN,NaN,NaN,NaN,NaN,spring rain -\r\nas the doctor speaks\r\ni thi...,en
5,5,spring moonset --,a rice ball for,breakfast,NaN,NaN,NaN,NaN,NaN,NaN,spring moonset --\r\na rice ball for\r\nbreakfast,en
6,6,sunny afternoon,an old man lingers,near the mailbox,NaN,NaN,NaN,NaN,NaN,NaN,sunny afternoon\r\nan old man lingers\r\nnear ...,en
7,7,cinco de mayo,horses roll,in the shallows,NaN,NaN,NaN,NaN,NaN,NaN,cinco de mayo\r\nhorses roll\r\nin the shallows,en
8,8,quitting time,the smell of rain,in the lobby,NaN,NaN,NaN,NaN,NaN,NaN,quitting time\r\nthe smell of rain\r\nin the l...,en
9,9,waves,slowly cresting towards shore,a faint moon,NaN,NaN,NaN,NaN,NaN,NaN,waves\r\nslowly cresting towards shore\r\na fa...,en
10,10,overnight rain --,the scent of orange blossoms,in a desert town,NaN,NaN,NaN,NaN,NaN,NaN,overnight rain --\r\nthe scent of orange bloss...,en
13,13,misty summer rain,calling pheasant,in Zen temple,NaN,NaN,NaN,NaN,NaN,NaN,misty summer rain\r\ncalling pheasant\r\nin Ze...,en
14,14,day is done,poppies amidst,the dying grass,NaN,NaN,NaN,NaN,NaN,NaN,day is done\r\npoppies amidst\r\nthe dying grass,en


In [34]:
df['haiku'] = df['haiku'].str.replace('\r\n', '\n')

In [39]:
window_size = 20
X = []
Y = []
for haiku in ' '*window_size + df['haiku'] + '\n':
    for i in range(window_size, len(haiku)):
        #print(haiku[i-20:i], '|||', haiku[i])
        X.append(haiku[i-window_size:i])
        Y.append(haiku[i])
    #break

In [43]:
len(X)

262446

In [17]:
#df['haiku_padded'] = df['haiku'].str.pad(150, side='right')

In [27]:
from keras.preprocessing.text import Tokenizer

'''t = Tokenizer(filters='', char_level=True)
t.fit_on_texts(df['haiku_padded'])
X = t.texts_to_sequences(df['haiku_padded'])'''

In [44]:
t = Tokenizer(filters='', char_level=True)
t.fit_on_texts(X)
X_int = t.texts_to_sequences(X)

In [47]:
Y_int = t.texts_to_sequences(Y)

In [57]:
len(t.word_counts)

65

In [58]:
X_r = np.reshape(X_int, (-1, window_size, 1))
X_r = X_r / float(len(t.word_counts))

In [64]:
Y_e = np_utils.to_categorical(Y_int)

In [65]:
Y_e.shape

(262446, 66)

In [66]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_r.shape[1], X_r.shape[2])))
model.add(Dense(Y_e.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [67]:
# define the checkpoint
filepath="storage/lstm-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [68]:
model.fit(X_r, Y_e, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
262446/262446 [==============================] - 72s 273us/step - loss: 3.0407

Epoch 00001: loss improved from inf to 3.04068, saving model to storage/lstm-01-3.0407.hdf5
Epoch 2/20
262446/262446 [==============================] - 66s 252us/step - loss: 2.9629

Epoch 00002: loss improved from 3.04068 to 2.96288, saving model to storage/lstm-02-2.9629.hdf5
Epoch 3/20
262446/262446 [==============================] - 66s 252us/step - loss: 2.8883

Epoch 00003: loss improved from 2.96288 to 2.88831, saving model to storage/lstm-03-2.8883.hdf5
Epoch 4/20
262446/262446 [==============================] - 66s 251us/step - loss: 2.8238

Epoch 00004: loss improved from 2.88831 to 2.82375, saving model to storage/lstm-04-2.8238.hdf5
Epoch 5/20
262446/262446 [==============================] - 66s 252us/step - loss: 2.7662

Epoch 00005: loss improved from 2.82375 to 2.76620, saving model to storage/lstm-05-2.7662.hdf5
Epoch 6/20
262446/262446 [==============================] - 66s 252us

In [72]:
[np.argmax(i) for i in model.predict(X_r[:10])]

[7, 5, 9, 6, 9, 8, 4, 15, 1, 7]

In [81]:
print(t.sequences_to_texts([[np.argmax(i) for i in model.predict(X_r[:1000])]])[0].replace('  ', '$').replace(' ', '').replace('$', ' '))

saroring say
--
th ea ew oaoetheee
ht   haohs suring sain
--the he sose  eff
r


en ae  s f toghts
suring saon
ng
--
thsoae serk soode
 h he n surmt rhturnoon
th the torhseaher

oh te he sarniin
sart  daotere
aar  g hn etin the sua ewo esute  ng theht
he suarl of tein
an the sosee sar r   ene  rorhe ng
theh 
e
hn we

n ar t saon
surr tne
 wamn --
the suent of thnene
satndem

an thcar he dhee
sarni rotmer
-ain
tote ng tethe ng  n tes shw se
sarerf sosn

ate eg th  es  he sast  ooons
sarteeng taouds
the saong ratt
e gn thcaoet s saohee
sarni in -h rmte
hssfee
otreweoter
fooe

ahe sart ro e
saeht cuiner
-
h the sarkeng fosh
atk
oght

aorhsh  e orn
ng
suwaoe tarest

n the suant ronet
ihe nsh e hf  suwaadl ror
a th r ne sh the saour  raoss ms
an r the seaetsark ng  sh e-a
the saote ng  doawew

he

on tyreee saodesaeh eerhr ie stwee sf the saont
in 
n t  e   eth wart
e
sarst saon 
  tshen
nh tee
 
ahe shd roon
satteeen  dayht-
-
-
s-. .
thseetet  torte
se ts



 t e shsse s haarece
nn
tsarni

In [84]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_r.shape[1], X_r.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(Y_e.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="storage/lstm-2layers-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X_r, Y_e, epochs=20, batch_size=64, callbacks=callbacks_list)

Epoch 1/20
262446/262446 [==============================] - 277s 1ms/step - loss: 2.9580

Epoch 00001: loss improved from inf to 2.95796, saving model to storage/lstm-2layers-01-2.9580.hdf5
Epoch 2/20
262446/262446 [==============================] - 265s 1ms/step - loss: 2.5818

Epoch 00002: loss improved from 2.95796 to 2.58181, saving model to storage/lstm-2layers-02-2.5818.hdf5
Epoch 3/20
262446/262446 [==============================] - 261s 993us/step - loss: 2.3647

Epoch 00003: loss improved from 2.58181 to 2.36466, saving model to storage/lstm-2layers-03-2.3647.hdf5
Epoch 4/20
262446/262446 [==============================] - 263s 1ms/step - loss: 2.2303

Epoch 00004: loss improved from 2.36466 to 2.23033, saving model to storage/lstm-2layers-04-2.2303.hdf5
Epoch 5/20
262446/262446 [==============================] - 262s 999us/step - loss: 2.1398

Epoch 00005: loss improved from 2.23033 to 2.13976, saving model to storage/lstm-2layers-05-2.1398.hdf5
Epoch 6/20
262446/262446 [====

In [85]:
print(t.sequences_to_texts([[np.argmax(i) for i in model.predict(X_r[:1000])]])[0].replace('  ', '$').replace(' ', '').replace('$', ' '))

aoaorial day
--
t stadow oor shrh
ooole soisteauring sain
--t  ihe soo on stiets
inthengsof tagics
auring sarn
n----
t cash farl nor ooeak est
aumde siternoon
t  old lan sesteri
ohwr the sarkeox
ainti da tay

tenee  tese in the stadoows
auitt ng shae
the stell of rain
on the sase

aile  tnewly trosh ng th erds
.tewe
t salmt roon
auercioht sain
--
the scent of andnge saossoms
tn t boatrt rhnc
aost  stnmer -ain
trll ng ooirrtnt in tua thaple
aay mf dane
aitrins a onst the so ng cooss
ailehing toouds
the soite raaals
of t baoeced aiosse
aorntain ttaeetsth  hh dialipeosks
iesss
the sare oase
aeght ofift
--tn the satk ng lan
thr sight

tostioar mooning
a land mietes
an the sun
e sase

to   shme io sa lunny say-aett raie tn the saorry blossoms
tner the sasleaarenhed an  --
the soosn nss chadow
ord

an trrfee clodsaevemy tea 
t sewee bo the saontain
aneeree  tith tittls
aorst daye 
h  snerithteien

the siw soon
aerinheeg tasteee

e
- . .
t  reseng tiase
statle
sn  
as   he r saaeection

aasta